In [12]:
import mujoco
from mujoco import MjModel, MjData, viewer
import time

# Load model and simulation state
model = MjModel.from_xml_path("pendulum.xml")
data = MjData(model)

# Function to run the simulation and render
def run_simulation(steps=500):
    # Launch viewer in a separate window
    viewer.launch_passive(model, data)
    
    # Step simulation
    for i in range(steps):
        # Optional: apply a simple torque
        data.ctrl[0] = 0.1  # constant torque
        mujoco.mj_step(model, data)
        time.sleep(0.01)    # slow down for visualization

# Run
run_simulation()


Exception in thread Thread-26 (_launch_internal):
Traceback (most recent call last):
  File "/home/yifeng/miniconda3/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/yifeng/work/env/mujoco/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "/home/yifeng/miniconda3/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yifeng/work/env/mujoco/lib/python3.10/site-packages/mujoco/viewer.py", line 380, in _launch_internal
    simulate = _Simulate(
RuntimeError: Unable to cast Python instance of type <class 'mujoco._structs.MjvCamera'> to C++ type '?' (#define PYBIND11_DETAILED_ERROR_MESSAGES or compile in debug mode for details)


KeyboardInterrupt: 